In [2]:
!pip install tensorflow opencv-python matplotlib
!pip install tf-slim
!pip install tensorflow-object-detection-api

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 26.0 MB/s  0:00:21 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 28.9 MB/s  0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 7.0 MB/s  0:00:00m 6.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 25.3 MB/s  0:00:00 eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 23.9 MB/s  0:00:006.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 24.3 MB/s  0:00:00 25.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 27.5 MB/s  0:00:02 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 28.2 MB/s  0:00:001.4 MB/s eta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 28.9 MB/s  0:00:

In [18]:
import tensorflow as tf
import numpy as np
import cv2
import os

In [16]:
# Paths
MODEL_DIR = "model/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8"
PATH_TO_SAVED_MODEL = os.path.join(MODEL_DIR, "saved_model")
IMAGE_PATH = "data/pexels-pixabay-47547.jpg"

In [12]:
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

In [13]:
# your test image
image_np = cv2.imread(IMAGE_PATH)
image_np_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

In [14]:
# Convert to tensor
input_tensor = tf.convert_to_tensor(image_np_rgb)
input_tensor = input_tensor[tf.newaxis, ...]

# Run detection
detections = detect_fn(input_tensor)

# Process output
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

detections

2025-08-12 12:19:51.878060: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 91097244 exceeds 10% of free system memory.
2025-08-12 12:19:51.892189: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 91097244 exceeds 10% of free system memory.
2025-08-12 12:19:52.005445: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 113891584 exceeds 10% of free system memory.
2025-08-12 12:19:52.097208: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 85675008 exceeds 10% of free system memory.
2025-08-12 12:19:54.047547: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 377318400 exceeds 10% of free system memory.


{'raw_detection_boxes': array([[0.04681766, 0.13317972, 0.98932964, 0.8524326 ],
        [0.11511902, 0.17741606, 0.96468747, 0.8773233 ],
        [0.10497619, 0.16921656, 0.9882369 , 0.87892395],
        [0.1265943 , 0.17206974, 0.97294515, 0.88484603],
        [0.06179649, 0.13051912, 0.989876  , 0.8528277 ],
        [0.06608913, 0.09656671, 0.98231703, 0.8051239 ],
        [0.12524857, 0.1538291 , 0.99271786, 0.8843285 ],
        [0.12667683, 0.18952343, 0.9668002 , 0.88247234],
        [0.08732235, 0.12469731, 0.99752516, 0.8804746 ],
        [0.10179542, 0.15336852, 0.9644013 , 0.87118477],
        [0.13995636, 0.18789724, 0.9686342 , 0.88758117],
        [0.10412571, 0.14997609, 0.9634476 , 0.87873834],
        [0.09266555, 0.11836029, 0.9687781 , 0.85931116],
        [0.10602921, 0.17404212, 0.9626626 , 0.88102967],
        [0.12553348, 0.19185564, 0.9628709 , 0.87587017],
        [0.13244236, 0.15481454, 0.9771715 , 0.8817059 ],
        [0.14496899, 0.24419288, 0.9730177 , 0.87

In [19]:
detection_classes = detections['detection_classes'].astype(int)  # Convert float to int class IDs
detection_scores = detections['detection_scores']

# Find the index of the highest score
top_index = detection_scores.argmax()

# Get the class ID of the top detection
top_class_id = detection_classes[top_index]

print("ID of the most relevant prediction:", top_class_id)

ID of the most relevant prediction: 17
